In [1]:
import os
os.chdir('..')

In [2]:
import time
import json
import os
import pickle
import copy
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, STATUS_FAIL

from algos import SBFGSABS
from algos import BFGS

from models import Nested
from models import MNL

data_folder = '../data/'
figures_folder = '../figures/'

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Preparing

First, we define the space of hyper-parameters optimization and we optimize the model with the non-stochastic algorithm.

In [3]:
space = {'window': hp.quniform('window', 1, 30, 1),
         'count_upd': hp.quniform('count_upd', 1, 20, 1),
         'factor_upd': hp.quniform('factor_upd', 1, 10, 0.1),
         'thresh_upd': hp.quniform('thresh_upd', 0, 10, 0.1)}

model = Nested(data_folder)

base_res = model.optimize(BFGS, **{'verbose': False, 'nbr_epochs': 50, 'thresh': 1e-6})

print("Number of epochs for non-stocha: {}".format(base_res.nep))

Number of epochs for non-stocha: 15


## Objective function

We define the objective function required for hyperopt.

In [7]:
def objective(model, algo, params, base_val, data_folder):
    win = params['window']
    cnt_upd = params['count_upd']
    fct_upd = params['factor_upd']
    thrsh_upd = params['thresh_upd']
    
    epochs = []
    fval = []
    for i in range(5):
        model = Nested(data_folder)
        
        res = model.optimize(algo,
                             **{'verbose': False, 'batch': 100, 'nbr_epochs': 50, 'thresh': 1e-6,
                                'thresh_upd': thrsh_upd,
                                'factor_upd': fct_upd, 'count_upd': cnt_upd, 'window': win})
        epochs.append(res.nep)
        fval.append(res.fun)

    if np.abs(np.mean(fval) - base_val) < 1e-5:
        status = STATUS_OK
    else:
        status = STATUS_FAIL

    return {'loss': np.max(epochs), 'value': np.mean(fval), 'status': status, 'epochs': epochs}


## Hyperopt

We use hyperopt to find the best hyperparameters for the ABS algorithm.

In [8]:
trials = Trials()
best = fmin(fn=lambda x: objective(model, SBFGSABS, x, base_res.fun, data_folder),
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

01-04-2019:16:48:43,281 INFO     [tpe.py:837] tpe_transform took 0.001128 seconds
01-04-2019:16:48:43,282 INFO     [tpe.py:867] TPE using 0 trials
01-04-2019:16:49:49,832 INFO     [tpe.py:837] tpe_transform took 0.001172 seconds
01-04-2019:16:49:49,833 INFO     [tpe.py:865] TPE using 1/1 trials with best loss 13.447991
01-04-2019:16:50:44,71 INFO     [tpe.py:837] tpe_transform took 0.004886 seconds
01-04-2019:16:50:44,72 INFO     [tpe.py:865] TPE using 2/2 trials with best loss 13.447991
01-04-2019:16:52:08,192 INFO     [tpe.py:837] tpe_transform took 0.002336 seconds
01-04-2019:16:52:08,194 INFO     [tpe.py:865] TPE using 3/3 trials with best loss 13.447991
01-04-2019:16:54:10,702 INFO     [tpe.py:837] tpe_transform took 0.001435 seconds
01-04-2019:16:54:10,702 INFO     [tpe.py:865] TPE using 4/4 trials with best loss 13.447991
01-04-2019:16:55:08,730 INFO     [tpe.py:837] tpe_transform took 0.001241 seconds
01-04-2019:16:55:08,730 INFO     [tpe.py:865] TPE using 5/5 trials with best 

RuntimeError: src/biogeme.cc:296: Biogeme exception: Error for data entry 0 : src/bioExprElem.cc:66: Biogeme exception: Invalid value for expression <(exp(((ASC_TRAIN[1] <0>+(B_TIME[3] <0>*TRAIN_TT_SCALED[34] <0>))+(B_COST[2] <0>*TRAIN_COST_SCALED[35] <0>)))^MU[4] <0>)>: nan

## After the optimization failed

We reload the model and try to run it again. 

In [9]:
model = Nested(data_folder)

base_res = model.optimize(BFGS, **{'verbose': False, 'nbr_epochs': 50, 'thresh': 1e-6})

RuntimeError: src/biogeme.cc:296: Biogeme exception: Error for data entry 75 : src/bioExprElem.cc:66: Biogeme exception: Invalid value for expression <(exp(((ASC_TRAIN[1] <75>+(B_TIME[3] <75>*TRAIN_TT_SCALED[34] <75>))+(B_COST[2] <75>*TRAIN_COST_SCALED[35] <75>)))^MU[4] <75>)>: nan